In [21]:
import numpy as np
import pandas as pd
import cv2

In [22]:
class_map = pd.read_csv('data/class_map.csv')
class_map.head()

,component_type,label,component
0,grapheme_root,0,ং
1,grapheme_root,1,ঃ
2,grapheme_root,2,অ
3,grapheme_root,3,আ
4,grapheme_root,4,ই


In [51]:
train_df = pd.read_csv('data/train.csv')
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [52]:
train_df[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8') # to save memory

In [53]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200840 entries, 0 to 200839
Data columns (total 5 columns):
image_id               200840 non-null object
grapheme_root          200840 non-null uint8
vowel_diacritic        200840 non-null uint8
consonant_diacritic    200840 non-null uint8
grapheme               200840 non-null object
dtypes: object(2), uint8(3)
memory usage: 3.6+ MB


In [54]:
train_df.drop('grapheme', axis= 1, inplace= True)

In [55]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic
0,Train_0,15,9,5
1,Train_1,159,0,0
2,Train_2,22,3,5
3,Train_3,53,2,2
4,Train_4,71,9,5


In [56]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200840 entries, 0 to 200839
Data columns (total 4 columns):
image_id               200840 non-null object
grapheme_root          200840 non-null uint8
vowel_diacritic        200840 non-null uint8
consonant_diacritic    200840 non-null uint8
dtypes: object(1), uint8(3)
memory usage: 2.1+ MB


In [29]:
train_imgs0 = pd.read_parquet("data/train_image_data_0.parquet")
train_imgs0.info()

,image_id,0,1,2,3,4,5,6,7,8,...,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331
0,Train_0,254,253,252,253,251,252,253,251,251,...,253,253,253,253,253,253,253,253,253,251
1,Train_1,251,244,238,245,248,246,246,247,251,...,255,255,255,255,255,255,255,255,255,254
2,Train_2,251,250,249,250,249,245,247,252,252,...,254,253,252,252,253,253,253,253,251,249
3,Train_3,247,247,249,253,253,252,251,251,250,...,254,254,254,254,254,253,253,252,251,252
4,Train_4,249,248,246,246,248,244,242,242,229,...,255,255,255,255,255,255,255,255,255,255


In [30]:
train_imgs1 = pd.read_parquet("data/train_image_data_1.parquet")
train_imgs1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50210 entries, 0 to 50209
Columns: 32333 entries, image_id to 32331
dtypes: object(1), uint8(32332)
memory usage: 1.5+ GB


In [31]:
train_imgs2 = pd.read_parquet("data/train_image_data_2.parquet")
train_imgs2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50210 entries, 0 to 50209
Columns: 32333 entries, image_id to 32331
dtypes: object(1), uint8(32332)
memory usage: 1.5+ GB


In [32]:
train_imgs3 = pd.read_parquet("data/train_image_data_3.parquet")
train_imgs3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50210 entries, 0 to 50209
Columns: 32333 entries, image_id to 32331
dtypes: object(1), uint8(32332)
memory usage: 1.5+ GB


In [33]:
train_imgs = pd.concat(
                    [train_imgs0, train_imgs1, train_imgs2, train_imgs3],
                    ignore_index= True)
train_imgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200840 entries, 0 to 200839
Columns: 32333 entries, image_id to 32331
dtypes: object(1), uint8(32332)
memory usage: 6.0+ GB


In [34]:
del train_imgs0, train_imgs1, train_imgs2, train_imgs3

In [35]:
img_id = train_imgs['image_id']

In [36]:
train_imgs.drop('image_id', axis= 1, inplace= True)

In [37]:
def resize(df, size=64):
    resized = {}
    for i in range(df.shape[0]):
        image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [38]:
train_imgs_resized = resize(train_imgs)

In [48]:
train_imgs_resized['image_id'] = img_id

In [57]:
train_df = train_df.merge(train_imgs_resized, on= 'image_id')
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,0,1,2,3,4,5,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,Train_0,15,9,5,253,252,253,253,254,253,...,252,252,252,252,253,253,253,253,253,253
1,Train_1,159,0,0,247,248,251,247,249,252,...,255,255,255,255,255,255,255,255,255,255
2,Train_2,22,3,5,251,248,253,251,254,252,...,253,253,251,251,251,252,252,253,253,252
3,Train_3,53,2,2,248,251,250,250,250,246,...,253,252,252,254,254,251,252,254,254,251
4,Train_4,71,9,5,248,245,238,240,237,234,...,255,255,255,255,255,255,255,255,255,255


In [58]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200840 entries, 0 to 200839
Columns: 4100 entries, image_id to 4095
dtypes: object(1), uint8(4099)
memory usage: 788.2+ MB


In [60]:
import pickle
pickle.dump(train_df, open('cleaned_training.p', 'wb'))